# 0. Goal, Purpose, and possible insights

The goal of this notebook is to find factors which influence daily sales, and how they do so. With the purpose of obtaining actionable insights which can be used to improve store sales or reduce costs

Ideas of possible insights
- Predict if a certain store is worth opening on a certain day
- Predict if a product family may be a worthy addition to a certain store
- Find cases in which a product family / store is under-peforming and could see gains from an investigation and taking actionable measures.
- Find the effectivity of product sales in increasing sales.


Ideas of insights that may be less cost-effective to gather:
- predict the increase/decrease on sales of a certain product family per stock per time: Analysis is complex, and of limited value since different products of the same family may behave differently.

Currently unusable/impossible insights:
- Find a baseline to compare how a normal product of a certain family is behaving, so that products can be compared to this baseline in the future: Unusable because we only have data aggregated per product family.
- Predict the necessary stock of a certain item (per store and product family): Impossible because we only have data aggregated per product family.


# 1. Setup

In [1]:
import data_preparation_attempt4 as data_prep
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FunctionTransformer

from hvplot import hvPlot, pandas #By wrapping a df with hvPlot you can easily call any plot type with type completion, pandas Allows hvplot graphs to work with dfs. Incluiding but not limited to using df.hvplot.graph_you_want
import holoviews as hv #All hvplots are returned inside a holoview layout. hv.Layout([plots_list]) can also be used to plot an array of plots together in one figure
import panel as pn #allow interactive widgets, for example to select which type of graphs to show, etc.
from bokeh.plotting import figure, show, output_notebook

#Reload all modules imported with %aimport every time before executing the Python code typed. https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload explicit
%aimport data_preparation_attempt4

pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns',1000)
pn.extension() #Allow showing panel objects inside the notebook, when using them with display() or as last element of the cell. To show them in the web just call your_panel_element.show()
output_notebook(hide_banner=True) #Allow bokeh to work inside notebooks

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
def describe_basics(table, number_of_examples=1) -> pd.DataFrame: #TODO add an example from head and one from tail to try to be more representative
      description_table = table.head(number_of_examples).T.add_prefix('example_')
      description_table['type'] = table.dtypes
      description_table['nulls'] = table.isna().sum()
      description_table['unique'] = table.nunique()
      description_table['total rows'] = len(table) 
      description_table.columns = description_table.columns.rename('columns')
      return description_table

def describe_categoricals(table, number_of_examples=1) -> pd.DataFrame:
      if len(table.columns)>0: 
            table = table.select_dtypes(['object', 'category', 'bool'])
            description_table = describe_basics(table, number_of_examples=number_of_examples)
            categorical_describe = table.describe(include=['object', 'datetime', 'datetimetz', 'timedelta', 'category']).T
            description_table['most_common'] = categorical_describe['top']
            description_table['freq_of_most_common'] = categorical_describe['freq']
            return description_table
      return None

def round_if_np_float(element, rounding_decimals):
    if np.issubdtype(type(element), np.floating):
        return element.round(rounding_decimals)
    return element

def describe_numerical_variable(variable:pd.Series, include_additional_examples=False, include_percentiles=False, decimals_for_calculations=2)->pd.Series:
    """
    Creates a pd series with commonly required descriptors for numerical variables.
    """
    column_meassures_as_row = { #TODO make it get all of this but min from a describe_variable_basics
        'variable':variable.name,
        'dtype':variable.dtype,
        'nulls':variable.isna().sum(),
        'unique':variable.nunique(),
        'total_rows':len(variable)
        #'min':variable.min(),
    }

    if include_additional_examples:
        column_meassures_as_row['first_value'] = variable[0],
        column_meassures_as_row['last_value'] = variable[len(variable)-1]
    
    if include_percentiles:
        #This conditional rounding is necessary because sometimes there isnt a clear sample at a certain quantile, and an average sample between the 2 that could be is used.
        column_meassures_as_row['25%'] = round_if_np_float(variable.quantile(0.25), decimals_for_calculations)
        column_meassures_as_row['50%'] = round_if_np_float(variable.quantile(0.5), decimals_for_calculations)
        column_meassures_as_row['75%'] = round_if_np_float(variable.quantile(0.75), decimals_for_calculations)
    
    column_meassures_as_row['max'] = variable.max()
    column_meassures_as_row['mean'] = variable.mean().round(decimals_for_calculations)
    column_meassures_as_row['std'] = variable.std().round(decimals_for_calculations)

    return pd.Series(column_meassures_as_row)

def describe_numericals(table_to_describe, include_additional_examples=False, include_percentiles=False, decimals_for_calculations=2):
    """Manually describe the numerical variables in pd.DataFrame
       This allows us to create an easy to read summary of that type of variables
    """
    rows = [describe_numerical_variable(table_to_describe[column_name]) for column_name in table_to_describe.select_dtypes(['number']).columns]
    return pd.concat(rows, axis=1).T #Carefully create a dataframe without modifying the displayed values

#Old method of describing numericals. Cant round. Tries to handle dates (but fails)
# def describe_numericals(table, number_of_examples=1, decimals_to_round=3) -> pd.DataFrame: 
#     table = table.select_dtypes(['number', 'datetime', 'datetimetz', 'timedelta'])

#     if len(table.columns)>0: 
#         description_table = describe_basics(table, number_of_examples)
#         numerical_describe = table.describe().T
#         description_table['mean'] = numerical_describe['mean']
#         description_table['std'] = numerical_describe['std']
#         description_table['min'] = numerical_describe['min']
#         description_table['25%'] = numerical_describe ['25%']
#         description_table['50%'] = numerical_describe['50%']
#         description_table['75%'] = numerical_describe['75%']
#         description_table['max'] = numerical_describe['max']
#         description_table = description_table

#         return description_table
    
#     return None

def display_table_description(table:pd.DataFrame, number_of_examples=1, decimals_for_calculations=2) ->None:
      display(describe_categoricals(table, number_of_examples))
      display(describe_numericals(table, decimals_for_calculations=decimals_for_calculations))

# 2. Get data

In [3]:
train_dataset = data_prep.get_train_dataset(100000, drop_sales=False)
daily_sales_data = train_dataset
stores_df = train_dataset.attrs['stores_df']
oil_df = train_dataset.attrs['oil_df']
transactions_df = train_dataset.attrs['transactions_df'] 
special_days_df = train_dataset.attrs['special_days_df']

# 3. Reasearch columns per table

## Sales Basic Data

### Check the basics and engineer/modify features as necessary, note any modificatiosn to the pipeline necessary for machine learning

### Check the basics

In [4]:
display_table_description(daily_sales_data)

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
date,2013-01-01,object,0,57,100000,2013-01-01,1782
family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_nbr,int64,0,54,100000,54,27.46,15.59
1,sales,float64,0,15007,100000,26067.0,191.79,652.0
2,onpromotion,int64,0,1,100000,0,0.0,0.0


### Rename columns for clarity and precision

In [5]:
def rename_sale_basics_data(daily_sales_data):
    daily_sales_data.rename(columns=
                            {'date':'date_of_sales',
                            'family':'product_family',
                            'store_nbr':'store_id',
                            'sales':'total_sales_of_day',
                            'onpromotion':'products_of_family_are_on_promotion_as_number'}, inplace=True)
    return daily_sales_data

daily_sales_data = rename_sale_basics_data(daily_sales_data)
display_table_description(daily_sales_data)

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
date_of_sales,2013-01-01,object,0,57,100000,2013-01-01,1782
product_family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_id,int64,0,54,100000,54,27.46,15.59
1,total_sales_of_day,float64,0,15007,100000,26067.0,191.79,652.0
2,products_of_family_are_on_promotion_as_number,int64,0,1,100000,0,0.0,0.0


### Check types, do transformations necessary and feature engineering necessary for data analysis. Write transformations/feature engineerings necessary for deep learning

#### Change date to date type

In [6]:
#Change date to date type
def correct_date_type(daily_sales_data):
    daily_sales_data.date_of_sales = pd.to_datetime(daily_sales_data.date_of_sales, format='%Y-%m-%d').dt.date.astype("datetime64[ns]")
    return daily_sales_data
daily_sales_data = correct_date_type(daily_sales_data)
display_table_description(daily_sales_data)

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
product_family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_id,int64,0,54,100000,54,27.46,15.59
1,total_sales_of_day,float64,0,15007,100000,26067.0,191.79,652.0
2,products_of_family_are_on_promotion_as_number,int64,0,1,100000,0,0.0,0.0


In [7]:
summary_df = daily_sales_data.describe()
summary_df['date_of_sales2'] = summary_df.date_of_sales
#Make sure any columns that were date tymes are converted back 
#for date_col in daily_sales_data.select_dtypes('datetime64[ns]'):
#    summary_df[date_col] = pd.to_datetime(summary_df[date_col], format='%Y-%m-%d').dt.date.astype("datetime64[ns]")
summary_df
#no se puede usar descrobe y roundear mas tarde porque count viene en describe y es numerico at least en dates. weird.
#SEGUIR DESDE ACA

,date_of_sales,store_id,total_sales_of_day,products_of_family_are_on_promotion_as_number,date_of_sales2
count,100000,100000.00000,100000.000000,100000.0,100000
mean,2013-01-28 13:25:21.792000256,27.46443,191.793399,0.0,2013-01-28 13:25:21.792000256
min,2013-01-01 00:00:00,1.00000,0.000000,0.0,2013-01-01 00:00:00
25%,2013-01-15 00:00:00,14.00000,0.000000,0.0,2013-01-15 00:00:00
50%,2013-01-29 00:00:00,27.00000,0.000000,0.0,2013-01-29 00:00:00
75%,2013-02-12 00:00:00,41.00000,84.000000,0.0,2013-02-12 00:00:00
max,2013-02-26 00:00:00,54.00000,26067.000000,0.0,2013-02-26 00:00:00
std,NaN,15.59037,652.003929,0.0,NaN


In [8]:
daily_sales_data.describe()

,date_of_sales,store_id,total_sales_of_day,products_of_family_are_on_promotion_as_number
count,100000,100000.00000,100000.000000,100000.0
mean,2013-01-28 13:25:21.792000256,27.46443,191.793399,0.0
min,2013-01-01 00:00:00,1.00000,0.000000,0.0
25%,2013-01-15 00:00:00,14.00000,0.000000,0.0
50%,2013-01-29 00:00:00,27.00000,0.000000,0.0
75%,2013-02-12 00:00:00,41.00000,84.000000,0.0
max,2013-02-26 00:00:00,54.00000,26067.000000,0.0
std,NaN,15.59037,652.003929,0.0


#### Engineer features from date

In [9]:
def add_new_date_features(daily_sales_data:pd.DataFrame):
    from pandas.tseries.offsets import MonthEnd
    daily_sales_data['day_of_year_of_sales'] = daily_sales_data.date_of_sales.dt.dayofyear #To analyze patterns that occur across a year
    daily_sales_data['day_of_month_of_sales'] = daily_sales_data.date_of_sales.dt.day #To analyze patterns that occur across a month
    #To analyze if being the last day of the month is relevant, keep in mind that it needs to be an additional feature since each month ends in a different day number:
    daily_sales_data['is_day_of_sales_last_day_of_month_number'] = (daily_sales_data.date_of_sales == daily_sales_data.date_of_sales + MonthEnd(1)).astype(int)
    daily_sales_data['is_day_of_sales_last_day_of_month'] = (daily_sales_data.is_day_of_sales_last_day_of_month_number == 1).astype(str)
    daily_sales_data['day_of_week_of_sales_as_number'] = daily_sales_data.date_of_sales.dt.dayofweek #To analyze patterns that occur across a week
    daily_sales_data['day_of_week_of_sales'] = daily_sales_data.date_of_sales.dt.day_name() #To assign the name to each day of week.
    return daily_sales_data
daily_sales_data = add_new_date_features(daily_sales_data)
display_table_description(daily_sales_data)

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
product_family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031
is_day_of_sales_last_day_of_month,False,object,0,1,100000,False,100000
day_of_week_of_sales,Tuesday,object,0,7,100000,Tuesday,14464


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_id,int64,0,54,100000,54,27.46,15.59
1,total_sales_of_day,float64,0,15007,100000,26067.0,191.79,652.0
2,products_of_family_are_on_promotion_as_number,int64,0,1,100000,0,0.0,0.0
3,day_of_year_of_sales,int32,0,57,100000,57,28.56,16.2
4,day_of_month_of_sales,int32,0,31,100000,31,14.68,8.36
5,is_day_of_sales_last_day_of_month_number,int64,0,1,100000,0,0.0,0.0
6,day_of_week_of_sales_as_number,int32,0,7,100000,6,3.0,2.0


In [10]:
def add_product_of_family_are_on_promotion(daily_sales_data:pd.DataFrame):
    """Add a add_product_of_family_are_on_promotion as true or false column instead of number """
    #As string, since graphing with boolean columns is problematic, and for ml you should use the number variant anyhow.
    daily_sales_data['products_of_family_are_on_promotion'] = (daily_sales_data.products_of_family_are_on_promotion_as_number == 1).astype('str') 
    display(daily_sales_data['products_of_family_are_on_promotion'].dtype)
    return daily_sales_data


daily_sales_data = add_product_of_family_are_on_promotion(daily_sales_data)
display_table_description(daily_sales_data)
daily_sales_data
#TODO Make products_on_family_are_on_promotion be displayed.

dtype('O')

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
product_family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031
is_day_of_sales_last_day_of_month,False,object,0,1,100000,False,100000
day_of_week_of_sales,Tuesday,object,0,7,100000,Tuesday,14464
products_of_family_are_on_promotion,False,object,0,1,100000,False,100000


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_id,int64,0,54,100000,54,27.46,15.59
1,total_sales_of_day,float64,0,15007,100000,26067.0,191.79,652.0
2,products_of_family_are_on_promotion_as_number,int64,0,1,100000,0,0.0,0.0
3,day_of_year_of_sales,int32,0,57,100000,57,28.56,16.2
4,day_of_month_of_sales,int32,0,31,100000,31,14.68,8.36
5,is_day_of_sales_last_day_of_month_number,int64,0,1,100000,0,0.0,0.0
6,day_of_week_of_sales_as_number,int32,0,7,100000,6,3.0,2.0


,date_of_sales,store_id,product_family,total_sales_of_day,products_of_family_are_on_promotion_as_number,day_of_year_of_sales,day_of_month_of_sales,is_day_of_sales_last_day_of_month_number,is_day_of_sales_last_day_of_month,day_of_week_of_sales_as_number,day_of_week_of_sales,products_of_family_are_on_promotion
id,,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,1,0,False,1,Tuesday,False
1,2013-01-01,1,BABY CARE,0.0,0,1,1,0,False,1,Tuesday,False
2,2013-01-01,1,BEAUTY,0.0,0,1,1,0,False,1,Tuesday,False
3,2013-01-01,1,BEVERAGES,0.0,0,1,1,0,False,1,Tuesday,False
4,2013-01-01,1,BOOKS,0.0,0,1,1,0,False,1,Tuesday,False
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2013-02-26,15,BREAD/BAKERY,157.0,0,57,26,0,False,1,Tuesday,False
99996,2013-02-26,15,CELEBRATION,0.0,0,57,26,0,False,1,Tuesday,False
99997,2013-02-26,15,CLEANING,759.0,0,57,26,0,False,1,Tuesday,False


#### Notes for ml feature engineering
- Engineering day of week, and day of month to be circular, instead of a range better reflects the nature of those variables, and will probably improve performance.
- Adding lag features for example yesterday, same day past week, same day past month, same day past year, could be important. Maybe even same day past cuarter.
- date of sales should probably be converted to either an absolute day since the start of time or even better a difference between the min date and that date.
- store_id should be OHE unless there is some meaning behind the order (could be the order they were founded! Verify!) 
- product_of_family_are_on_promotion should probably be transformed to int32 since its a true or false value.
-

In [11]:
def create_sales_basic_data_pipeline():
    pipeline = Pipeline([
        ('rename_columns', FunctionTransformer(rename_sale_basics_data)),
        ('correct_date_type', FunctionTransformer(correct_date_type)),
        ('add_new_date_features', FunctionTransformer(add_new_date_features))
    ])
    return pipeline

daily_sales_data = create_sales_basic_data_pipeline().fit_transform(daily_sales_data)

In [12]:
display_table_description(daily_sales_data)

columns,example_0,type,nulls,unique,total rows,most_common,freq_of_most_common
product_family,AUTOMOTIVE,object,0,33,100000,AUTOMOTIVE,3031
is_day_of_sales_last_day_of_month,False,object,0,1,100000,False,100000
day_of_week_of_sales,Tuesday,object,0,7,100000,Tuesday,14464
products_of_family_are_on_promotion,False,object,0,1,100000,False,100000


,variable,dtype,nulls,unique,total_rows,max,mean,std
0,store_id,int64,0,54,100000,54,27.46,15.59
1,total_sales_of_day,float64,0,15007,100000,26067.0,191.79,652.0
2,products_of_family_are_on_promotion_as_number,int64,0,1,100000,0,0.0,0.0
3,day_of_year_of_sales,int32,0,57,100000,57,28.56,16.2
4,day_of_month_of_sales,int32,0,31,100000,31,14.68,8.36
5,is_day_of_sales_last_day_of_month_number,int64,0,1,100000,0,0.0,0.0
6,day_of_week_of_sales_as_number,int32,0,7,100000,6,3.0,2.0


### Analyze basic variable interactions of interest to the analysis, and write insights

In [13]:
def graph_target_by_categorical(data_source:pd.DataFrame, cat_variable_name:str|tuple[str,str], target_variable_name:str, add_ref_line:bool=False) ->tuple[pd.DataFrame, hv.Layout]:
    #If the variable name is a tuple, it indicates we want to graph the first element, but ordered by the second one, so we require both on group by.
    #The typical use case is days of week, month of year, etc, where we want to graph by name but order by number.
    #Be careful the categorifcal variable used must refer to a string column, if its a boolean column you will run into an odd exception.
    if type(cat_variable_name) == tuple:
        variables_to_group_by = [cat_variable_name[0]]#, cat_variable_name[1]]
        variable_to_graph = cat_variable_name[0]
        variable_to_sort_by = cat_variable_name[0] #cat_variable_name[1]
    else:
        variables_to_group_by = cat_variable_name
        variable_to_graph = cat_variable_name
        variable_to_sort_by = cat_variable_name


    agg_target_variable_name = f'avg_{target_variable_name}'
    grouped_df = data_source.groupby(variables_to_group_by,as_index=False
                            ).agg({target_variable_name: 'mean'}
                            ).sort_values(variable_to_sort_by
                            ).rename(columns={target_variable_name:agg_target_variable_name})
    
    grouped_bar_plot = hvPlot(grouped_df).bar(x=variable_to_graph, y=agg_target_variable_name)
    
    #if add_ref_line:
    #    grouped_df[f'grand_{agg_target_variable_name}'] = grouped_df[agg_target_variable_name].mean() #For creating the reference line
    #    avg_reference_line = hvPlot(grouped_df).line(x=variable_to_graph, y=f'grand_{agg_target_variable_name}', color='gray',line_dash='dashed')
    #    grouped_bar_plot *= avg_reference_line #Add the reference line to the original plot
    
    return grouped_df, grouped_bar_plot

def create_numerical_targets_by_categoricals_graph(data_source:pd.DataFrame, cat_variable_names:list[str|tuple[str,str]], numerical_target_variable_names:list[str], add_ref_line:bool=False)->None:
    plots = []
    for target_variable in numerical_target_variable_names:
        for cat_variable in cat_variable_names:
            grouped_df, grouped_plot  = graph_target_by_categorical(data_source, cat_variable, target_variable, add_ref_line)
            plots.append(grouped_plot)
    
    return hv.Layout(plots).cols(2)

cat_variables_to_split_by = ['store_id', ('products_of_family_are_on_promotion', 'products_of_family_are_on_promotion_as_number'), 'day_of_year_of_sales', 'day_of_month_of_sales', ('day_of_week_of_sales','day_of_week_of_sales_as_number'), 'is_day_of_sales_last_day_of_month']
numerical_target_variables = ['total_sales_of_day']

graphs_with_line_dict = {True: create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, True),
                         False: create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, False)}

plot_reference_line_selector = pn.widgets.Select(name='Use Line', options=[False, True])

@pn.depends(plot_reference_line_selector.param.value)
def display_correct_plots(add_reference_line_selector:bool):
    print(f'add_reference_line_selector{add_reference_line_selector}')
    return create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, add_reference_line_selector)
    #return graphs_with_line_dict[add_reference_line_selector] #alt to cache.


create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, False)


#layout = pn.Column(plot_reference_line_selector, display_correct_plots(True))

#display(layout)
#display(layout.servable()) #Doesnt update bokeh graphs, but layout.show does work.

:Layout
   .Bars.I   :Bars   [store_id]   (avg_total_sales_of_day)
   .Bars.II  :Bars   [products_of_family_are_on_promotion]   (avg_total_sales_of_day)
   .Bars.III :Bars   [day_of_year_of_sales]   (avg_total_sales_of_day)
   .Bars.IV  :Bars   [day_of_month_of_sales]   (avg_total_sales_of_day)
   .Bars.V   :Bars   [day_of_week_of_sales]   (avg_total_sales_of_day)
   .Bars.VI  :Bars   [is_day_of_sales_last_day_of_month]   (avg_total_sales_of_day)

In [31]:
def make_categorical_variable_influence_table(df:pd.DataFrame, cat_var_name:str, num_var_name:str, agg_type:str='mean', make_categorical_variable_name_a_column=True):
    """ """
    #TODO hacer que anada la numerical variable como columna para que sea facilmente concatenable ?
    agg_num_var_name = f'{agg_type}_{num_var_name}'
    avg_by_cat = df.groupby(cat_var_name, as_index=False
                   ).agg({num_var_name:agg_type}
                   ).rename(columns={num_var_name:agg_num_var_name})

    #Make sure the categorical variable is a string (even if it was numerical)
    avg_by_cat[cat_var_name] = avg_by_cat[cat_var_name].astype(str)
    
    #Calculate the mean for all other categories
    means_of_others = []
    for category in avg_by_cat[cat_var_name]:
        is_other_category = avg_by_cat[cat_var_name]!=category
        mean_of_others = avg_by_cat[is_other_category][agg_num_var_name].mean()
        means_of_others.append(mean_of_others)
        

    avg_by_cat[f'{agg_num_var_name}_of_others'] = means_of_others
    avg_by_cat[f'diff_with_{agg_num_var_name}_of_others'] = avg_by_cat[agg_num_var_name] - avg_by_cat[f'{agg_num_var_name}_of_others']
    avg_by_cat[f'abs_diff_with_{agg_num_var_name}_of_others'] = np.abs(avg_by_cat[f'diff_with_{agg_num_var_name}_of_others'])


    #avg_by_cat[f'diff_with_{agg_num_var_name}_of_others_is_%_of_value'] = round(avg_by_cat[f'diff_with_{agg_num_var_name}_of_others'] / avg_by_cat[agg_num_var_name] * 100, 2)
    #avg_by_cat[f'diff_with_{agg_num_var_name}_of_others_is_%_of_other_value'] = round(avg_by_cat[f'diff_with_{agg_num_var_name}_of_others'] / avg_by_cat[f'{agg_num_var_name}_of_others'] * 100, 2)
    #avg_by_cat[f'abs_diff_with_{agg_num_var_name}_of_others_is_%_of_other_value'] = abs(avg_by_cat[f'diff_with_{agg_num_var_name}_of_others_is_%_of_other_value'])
    #Fruta creo, no puedo restar un coso y dividir por el otro es ridiculo.
    #avg_by_cat[f'diff_with_{agg_num_var_name}_of_others_is_%_of_avg'] = round(avg_by_cat[f'diff_with_{agg_num_var_name}_of_others'] / np.average(avg_by_cat[agg_num_var_name]) * 100, 2)

    if make_categorical_variable_name_a_column:
        avg_by_cat.insert(0, 'influencer_variable', cat_var_name) #add a column name cat var first
        avg_by_cat = avg_by_cat.rename(columns={cat_var_name:'category'}) #rename the column wihth the category to category.
        avg_by_cat.insert(0, 'influencer', avg_by_cat['influencer_variable'] + ' = ' +avg_by_cat['category'])

    return avg_by_cat

make_categorical_variable_influence_table(daily_sales_data, 'day_of_week_of_sales', 'total_sales_of_day')

,influencer,influencer_variable,category,mean_total_sales_of_day,mean_total_sales_of_day_of_others,diff_with_mean_total_sales_of_day_of_others,abs_diff_with_mean_total_sales_of_day_of_others
0,day_of_week_of_sales = Friday,day_of_week_of_sales,Friday,179.085549,194.018976,-14.933427,14.933427
1,day_of_week_of_sales = Monday,day_of_week_of_sales,Monday,175.851245,194.558027,-18.706781,18.706781
2,day_of_week_of_sales = Saturday,day_of_week_of_sales,Saturday,249.753755,182.240942,67.512814,67.512814
3,day_of_week_of_sales = Sunday,day_of_week_of_sales,Sunday,246.593821,182.767597,63.826224,63.826224
4,day_of_week_of_sales = Thursday,day_of_week_of_sales,Thursday,159.450359,197.291508,-37.841149,37.841149
5,day_of_week_of_sales = Tuesday,day_of_week_of_sales,Tuesday,147.543942,199.275911,-51.731969,51.731969
6,day_of_week_of_sales = Wednesday,day_of_week_of_sales,Wednesday,184.920734,193.046445,-8.125711,8.125711


In [32]:
def make_key_influencers_table(df, cat_variables_names, numerical_target, agg='mean'):
    influence_tables = []
    for cat_variable_name in cat_variables_names:
        influence_tables.append(make_categorical_variable_influence_table(df, cat_variable_name, numerical_target, agg))
    
    #Ridiculously enough, pd.concat(influence_tables, ignore_index=True, axis=0) doesn't seem to work.
    return pd.DataFrame.from_records(np.concat([influence_table.to_records(index=False) for influence_table in influence_tables])).sort_values(f'diff_with_{agg}_{numerical_target}_of_others', ascending=False)
    #'diff_with#
    #.sort_values(f'abs_diff_with_{agg}_{numerical_target}_of_others_is_%_of_other_value', ascending=False)

def display_styled_table(df, include_replacing_underscores=False, decimals_to_show=1):
    
    if include_replacing_underscores: #Careful its probably extremely bad for performance since we are copiying the df, but otherwise we'd replace the original column names .
        #To be frank, its probably best practice to just use column names wiht spaces. 
        df2 = pd.DataFrame(df)
        df2.columns = df2.columns.str.replace("[_]", " ", regex=True) #Might be avoidable with https://stackoverflow.com/a/58157489/14301911
        display(df2.style.set_table_styles( [dict(selector="th", props=[('text-align', 'center'), ('max-width', '100px')])]
                                                        ).hide(axis='index'
                                                        ).format(precision=decimals_to_show, thousands=' '
                                                        ).set_properties(**{'text-align': 'center'})
        )

    else:
        display(df.style.set_table_styles( [dict(selector="th", props=[('text-align', 'center'), ('max-width', '100px'),  ('word-break', 'break-all')])]
                                                            ).hide(axis='index'
                                                            ).format(precision=decimals_to_show, thousands=' '
                                                            ).set_properties(**{'text-align': 'center'})
    
        )

#example
ex_key_influencers_table = make_key_influencers_table(daily_sales_data, ['day_of_week_of_sales', 'day_of_month_of_sales'], 'total_sales_of_day')
display_styled_table(ex_key_influencers_table.head(10), True)


influencer,influencer variable,category,mean total sales of day,mean total sales of day of others,diff with mean total sales of day of others,abs diff with mean total sales of day of others
day_of_month_of_sales = 2,day_of_month_of_sales,2,284.8,187.5,97.3,97.3
day_of_week_of_sales = Saturday,day_of_week_of_sales,Saturday,249.8,182.2,67.5,67.5
day_of_week_of_sales = Sunday,day_of_week_of_sales,Sunday,246.6,182.8,63.8,63.8
day_of_month_of_sales = 27,day_of_month_of_sales,27,241.5,188.9,52.6,52.6
day_of_month_of_sales = 3,day_of_month_of_sales,3,237.9,189.1,48.8,48.8
day_of_month_of_sales = 6,day_of_month_of_sales,6,236.0,189.1,46.9,46.9
day_of_month_of_sales = 16,day_of_month_of_sales,16,225.6,189.5,36.1,36.1
day_of_month_of_sales = 5,day_of_month_of_sales,5,224.1,189.5,34.5,34.5
day_of_month_of_sales = 26,day_of_month_of_sales,26,222.9,189.6,33.3,33.3
day_of_month_of_sales = 13,day_of_month_of_sales,13,220.0,189.7,30.4,30.4


In [125]:
def key_influencers_vertical_graph(key_influencers_table, num_var_name, agg_used): 
    """By far the easiest for visualizing a large number of categories with references."""
    #print(750/(len(key_influencers_table)))
    return hvPlot(key_influencers_table).barh(x='influencer', y=f'diff_with_{agg_used}_{num_var_name}_of_others', height=18*len(key_influencers_table) + 150)

def key_influencers_double_graph(key_influencers_table, num_var_name, agg_used):
    abs_influence_name = f'diff_with_{agg_used}_{num_var_name}_of_others'
    #display(key_influencers_table.query(f" {abs_influence_name} > 0"))

    negatives_plot = hvPlot(key_influencers_table.query(f" {abs_influence_name} < 0")
                  ).barh(x='influencer', y=f'diff_with_{agg_used}_{num_var_name}_of_others', height=18*len(key_influencers_table) + 150
                  ).opts(color='tomato', axiswise=True)

    positives_plot = hvPlot(key_influencers_table.query(f" {abs_influence_name} > 0")
                  ).barh(x='influencer', y=f'diff_with_{agg_used}_{num_var_name}_of_others', height=18*len(key_influencers_table) + 150
                  ).opts(invert_yaxis=True, legend_opts={'location':'right'}, axiswise=True) #yaxis_location='right',

    bokeh_fig:bokeh.plotting._figure.figure = hv.render(negatives_plot)
    print(bokeh_fig)
    bokeh_fig.title = 'Test'
    print(bokeh_fig.descriptors())
    print(bokeh_fig.dataspecs())
    show(bokeh_fig.clone())
    bokeh_fig.select(type=object).update(title="Asd")
    print()
    
    #ideally i'd want to be able to call figure wrapping the current figure.
    #figure(bokeh_fig) #but this doesnt work

    #figure(bokeh_fig)
    #figure(title='test', height=100)

    #bokeh_fig.glyph.y_axis_location='right'
    #display(bokeh_fig)
    #bokeh.plotting.
    #bokeh.plotting.curplot(hv.render(negatives_plot))

    #print(type(hv.render(negatives_plot)))
    #bokeh_plott = hv.render(negatives_plot)

    #print(bokeh_plott)
    #fig = figure(y_axis_location='right')
    #fig.render(negatives_plot)
    #bokeh_plott.y_axis_location='right'
    #p = figure()
    return bokeh_fig

    #return negatives_plot + positives_plot

double_graph = key_influencers_double_graph(ex_key_influencers_table, 'total_sales_of_day', 'mean')
#Asi se puede usar para todas las variables de la table, que podria ser una sola si queres.
display(show(double_graph))
#display(key_influencers_vertical_graph(ex_key_influencers_table, 'total_sales_of_day', 'mean'))

figure(id='27b3fa4c-109c-49e4-97a0-d4386265d7e2', ...)
[<bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1C29310>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1F20550>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1F205D0>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1F20610>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1F20650>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C1F20690>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C22E5610>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C22E5690>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C22E5710>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C22E5750>, <bokeh.core.property.descriptors.PropertyDescriptor object at 0x000001F6C22E5790>, <bokeh.core.property.descriptor

TypeError: '_list_attr_splat' object is not callable

In [35]:
def key_influencer_variables_table(key_influencers_table, num_var_name, agg_used, flatten_index=True):
    """Create a table showing the min, mean, and max diff each category of a variable has with the rest of the categories
       Variables with a higher mean are most influential. 
    """
    df = key_influencers_table.groupby('influencer_variable', as_index=False).agg({f'abs_diff_with_{agg_used}_{num_var_name}_of_others':['min', 'mean','max']})
    if flatten_index:
        df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df
key_influencer_variables = key_influencer_variables_table(ex_key_influencers_table, 'total_sales_of_day', 'mean')

def key_influencer_variables_graph(flat_index_key_influencer_variables, num_var_name, agg_used):
    """HvPlot cant handle not flat indexes"""
    num_avg_col_name = f'abs_diff_with_{agg_used}_{num_var_name}_of_others mean' #Could be interesting to graph also min and max, sicne they are available. TBH possibly only max.

    influencer_variables_plot = hvPlot(flat_index_key_influencer_variables).barh(x='influencer_variable', y=num_avg_col_name, height=18*len(flat_index_key_influencer_variables) + 150)
    return influencer_variables_plot

display(key_influencer_variables_graph(key_influencer_variables, 'total_sales_of_day', 'mean'))


:Bars   [influencer_variable]   (abs_diff_with_mean_total_sales_of_day_of_others mean)

In [ ]:
df_dict = {True: create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, True),
        False: create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, False)}

#plot_reference_line_selector = pn.widgets.Select(name='df_to_use', options=list(df_dict.keys()))

@pn.depends(plot_reference_line_selector.param.value)
def show_graph_based_on_selection(selector_value):
    return df_dict[selector_value]
#    return pn.Row(df_dict[selector_value])
    #return pn.widgets.DataFrame(df_dict[selector_value].head(5)) 

#layout = pn.Column(plot_reference_line_selector, show_df)
#layout

In [ ]:
plot_reference_line_selector = pn.widgets.Select(name='df_to_use', options=list(df_dict.keys()))

@pn.depends(plot_reference_line_selector.param.value)
def show_graph_based_on_selection(selector_value):
    #display()
    return df_dict[selector_value]
    #return selector_value

widget_col = pn.Column(plot_reference_line_selector, show_graph_based_on_selection)
widget_col

widget_col.show()
#nothing = show_df(True)

#display(nothing)

In [ ]:

#create_numerical_targets_by_categoricals_graph(daily_sales_data, cat_variables_to_split_by, numerical_target_variables, add_ref_line=False)

# Backup